### Create the environment
To do so, we select an API endpoint and collect a token from our configuration. <br>
We also need an access token, in this example we use [oidc-agent](https://github.com/indigo-dc/oidc-agent#:~:text=oidc%2Dagent%20is%20a%20set,session%20or%20a%20login%20session.) to get one. <br>

In [ ]:
eosc_perf_api="https://perf.test.fedcloud.eu/api/v1"
access_token=$(oidc-token egi-prod)

### (Conditional) Register, if not done already
To use our service as user, first we need to accept the terms of usage and register. <br>
Make sure to read the [terms and conditions](https://performance.services.fedcloud.eu/privacy_policy).

In [ ]:
curl -X 'POST' \
  "$eosc_perf_api/users:register" \
  -H "Authorization: Bearer $access_token"

### Search for the benchmark id that produced our result
You can get a list of all available benchmarks using GET /benchmarks
> Benchmarks are public to the Internet, access token is not needed

In [ ]:
benchmarks=$(curl -X 'GET' "$eosc_perf_api/benchmarks?docker_image=deephdc/deep-oc-benchmarks_cnn&docker_tag=benchmark")
benchmark=$(echo $benchmarks | jq '.items[0]')
echo $benchmark | jq '.json_schema = "..."'

### Search for the flavor id used to run our benchmark
First you need to find the site where the benchmark was run. <br>
Once the site id is collected, it is possible to access and select the site flavors. <br>
> In this example we will use ':search' endpoint to find the flavor

In [ ]:
sites=$(curl -X 'GET' "$eosc_perf_api/sites?name=ForHLR2")
site=$(echo $sites | jq '.items[0]')
echo $site | jq

In [ ]:
site_id=$(echo $site | jq -r '.id')
flavors=$(curl -X 'GET' "$eosc_perf_api/sites/$site_id/flavors:search?terms=haicore-gpu8")
flavor=$(echo $flavors | jq '.items[0]')
echo $flavor | jq

### Search for the tags to relate your result
Collect the tags you want to link to your result so users can find it easily. <br>
> If you do not know the name of the tag you can use :search as generic filter.

In [ ]:
tag_gpu=$(curl -X 'GET' "$eosc_perf_api/tags?name=gpu" | jq '.items[0]')
echo $tag_gpu | jq

### Upload your result
Use an execution datetime and the collected site_id and flavor_id to upload a result. <br>

In [ ]:
execution_datetime="2021-09-08T20:37:10.192459"
benchmark_id=$(echo $benchmark | jq -r '.id')
flavor_id=$(echo $flavor | jq -r '.id')
tag_gpu_id=$(echo $tag_gpu | jq -r '.id')
result_json='{"machine_config": {"cpu_info": {"num_cores": 256, "cpu_info": "AMD EPYC 7742 64-Core Processor", "mhz_per_cpu": 3313.455}, "gpu_info": {"count": 8, "model": "A100-SXM4-40GB", "memory": 39420913408}, "memory_total": 1082004615168, "memory_available": 1061217103872}, "benchmark": {"version": "0.6.0", "flavor": "dataset", "docker_base_image": "nvcr.io/nvidia/tensorflow:20.06-tf2-py3", "dataset": "imagenet_mini", "tf_version": "2.2.0"}, "training": {"allow_growth": true, "data_format": "NCHW", "device": "gpu", "local_parameter_device": "cpu", "models": ["googlenet", "inception3", "resnet50", "vgg16"], "num_gpus": 4, "optimizer": "sgd", "use_fp16": false, "variable_update": "parameter_server", "googlenet": {"batch_size": 3840, "batch_size_per_device": 960, "num_batches": 100, "num_epochs": 0.0, "average_examples_per_sec": 4965.329768107184, "execution_time_sec": 76.60696387290955}, "inception3": {"batch_size": 960, "batch_size_per_device": 240, "num_batches": 100, "num_epochs": 0.0, "average_examples_per_sec": 1316.101905705066, "execution_time_sec": 72.25684809684753}, "resnet50": {"batch_size": 960, "batch_size_per_device": 240, "num_batches": 100, "num_epochs": 0.0, "average_examples_per_sec": 1607.2773590283105, "execution_time_sec": 59.14579105377197}, "vgg16": {"batch_size": 640, "batch_size_per_device": 160, "num_batches": 100, "num_epochs": 0.0, "average_examples_per_sec": 642.5885371362209, "execution_time_sec": 98.82184600830078}, "score": 8531.297569976781}, "global_start_time": "2021-02-23T12:19:07.650951Z", "global_end_time": "2021-02-23T12:26:49.507629Z", "global_execution_time_sec": 461.8566780090332}'

In [ ]:
query="execution_datetime=$execution_datetime&benchmark_id=$benchmark_id&flavor_id=$flavor_id&tags_ids=$tag_gpu_id"
curl -X 'POST' "$eosc_perf_api/results?$query" \
  -H 'accept: application/json' \
  -H "Authorization: Bearer $access_token" \
  -H 'Content-Type: application/json' \
  -d "$result_json" | jq